In [2]:
#In case you don't have qiskit, install it now
%pip install qiskit --quiet
%pip install qiskit-aer --quiet
#Installing/upgrading pylatexenc seems to have fixed my mpl issue
#If you try this and it doesn't work, try also restarting the runtime/kernel
%pip install pylatexenc --quiet


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [3]:

#Let's go ahead and import all this stuff too
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
import numpy as np
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
%matplotlib inline

## Game Code

In [4]:
# initialize quantum circuits
def init_qcs(num_players):
    # night circuit
    night_qc = QuantumCircuit(num_players, num_players)
    # voting circuit
    vote_qc = QuantumCircuit(num_players, num_players)
    return night_qc, vote_qc


In [5]:
# set up the night circuit, everyone is alive
def init_night(night_qc, num_players):
    night_qc.x(range(num_players))

# choose random player to be imposter
def choose_imposter(num_players):
    return np.random.randint(num_players)

In [6]:
# round 1 tell everyone their roles
def round_1(num_players, imposter):
    for i in range(num_players):
        if i == imposter:
            n = input(f"Player {i}: YOU ARE THE IMPOSTER")
            input("Please pass the device to the next player")
        else:
            n = input(f"Player: {i}: You are a crewmate")
            input("Please pass the device to the next player")

In [7]:
# get the actions of each player for the night round
def get_actions(num_players, imposter, alive_players):
    victim = None
    for i in range(num_players):
        if (alive_players[i] == 0):
            continue
        if i == imposter:
            choice = 0
            while choice != 1 and choice != 2:
                try:
                    choice = int(input(f"Player {i}: You are the imposter. Do you want to kill 1 or entangle with 2 players? Enter 1 or 2"))
                except:
                    continue
                if choice == 1:
                    victim = imposter_kill1(num_players, imposter)
                elif choice == 2:
                    victim = imposter_kill2(num_players, imposter)
        else:
            for j in range(2):
                n = None
                target = np.random.randint(10)
                while n is None:
                    try:
                        n = int(input(f"Player {i}: please type {target}"))
                        if (n == target):
                            break
                        else:
                            n = None
                    except:
                        n = None
        input("Please pass the device to the next player")
    return victim

# kill singular victim of the imposter
def imposter_kill1(num_players, imposter):
    victim = None
    while victim is None:
        n = input("Who do you want to kill?")
        if n == "q": break
        n = int(n)
        if n != imposter and isinstance(n,int) and n < num_players:
            victim = int(n) 
    return victim

# entangle two victims of the imposter
def imposter_kill2(num_players, imposter):
    victim = []
    while len(victim) < 1:
        n = input("Who do you want to entangle with 1?")
        if n == "q": break
        n = int(n)
        if n != imposter and isinstance(n, int) and n < num_players:
            victim.append(int(n))
    while len(victim) < 2:
        n = input("Who do you want to entangle with 2?")
        if n == "q": break
        n = int(n)
        if n != imposter and n < num_players and n != victim[0]:
            victim.append(int(n))
    return victim

def night_round(night_qc, num_players, imposter, alive_players):
    victim = get_actions(num_players, imposter, alive_players)
    # imposter kills victim
    if isinstance(victim, int):
        night_qc.x(victim)
    else:
        night_qc.h(imposter)
        night_qc.cx(imposter, victim[0])
        night_qc.cx(imposter, victim[1])

# measure the night circuit and count the results
def measure_night(qc, num_players):
    qc.measure(range(num_players), range(num_players))
    backend = Aer.get_backend('qasm_simulator')
    night_counts = backend.run(qc, shots=1024).result().get_counts(qc)
    print(night_counts)
    plot_histogram(night_counts)
    return night_counts

# return the most common result of measurement
def most_common(night_counts):
    return max(night_counts, key=night_counts.get)

# determine who died and if the imposter was killed
def who_died(night_qc, num_players, imposter, alive_players):
    night_counts = measure_night(night_qc, num_players)
    states = str(most_common(night_counts))
    states = states[::-1]
    for i in range(len(states)):
        if states[i] == "0":
            print(f"Player {i} died")
            alive_players[i] = 0
            if i == imposter:
                print("The imposter was killed!")
                return False
    return True

# the circuit used to keep track of voting. helper method for vote. 
def voting_circuit(num_players, imposter, sabotages_left, alive_players):
    vc = QuantumCircuit(num_players,num_players)
    vc.barrier()
    vc.h(range(num_players))
    # ask each player for their selection
    for i in range(num_players):
        if (alive_players[i] == 0):
            continue
        n = input(f"Player {i}, enter who you think the imposter is: ")
        if (i != imposter):
            vc.p(np.pi / num_players, int(n))
        else:
            if (sabotages_left > 0):
                sabotage = input("Do you want to sabotage a player? (y/n)")
                if (sabotage == "y"):
                    sabotage = True
                    sabotages_left -= 1
                else:
                    sabotage = False

    vc.barrier()
    
    # sabotage random alive player. 
    if (sabotage):
        sabotaged_player = np.random.randint(num_players)
        while (alive_players[sabotaged_player] != 1):
            sabotaged_player = np.random.randint(num_players)
        vc.p(np.pi / num_players, sabotaged_player)
        
    vc.h(range(num_players))
    vc.barrier()
    vc.measure(range(num_players), range(num_players))
    vc.draw('mpl')
    return vc

# Return the player to kill based on the voting circuit
def vote(num_players, imposter, sabotages_left, alive_players):
    vc = voting_circuit(num_players, imposter, sabotages_left, alive_players)
    backend=Aer.get_backend('qasm_simulator')
    job = backend.run(vc, shots=1024)
    counts = job.result().get_counts(vc)
    plot_histogram(counts)
    
    # get the bitstring found the most
    best = None
    most = 0
    for bitstring, c in counts.items():
        if best==None or c>most and ('1' in bitstring):
            most = c
            best = bitstring

    print(f"Solution is {best} with a count of {most}")

    index = 0
    while (best[index] != '1'):
        index = index + 1

    to_kill = num_players - index - 1
    print(f"Player {to_kill} has been killed! ") 
    alive_players[to_kill] = 0
    return to_kill

# check if the game is over
def check_game_over(alive_players, imposter):
    num_alive = 0
    for i in range(len(alive_players)):
        # imposter is dead
        if i == imposter and alive_players[i] == 0:
            print("The imposter was killed!")
            print("Crewmates wins!")
            return True
        if alive_players[i] == 1:
            num_alive += 1
    # imposter last alive
    if num_alive == 1:
        print("All the crewmates are dead!")
        print("Imposter wins!")
        return True
    return False

## Player interface

In [13]:
# run this cell to start the game and choose the number of players

num_players = int(input("how many players do you have? "))
alive_players = [1 for i in range(num_players)] 
sabotages_left = int(np.floor(num_players / 3))
print("You have", sabotages_left, "sabotages left.")

imposter_alive = True
imposter = choose_imposter(num_players)
alive_players = [1 for i in range(num_players)]
round_1(num_players, imposter)

ValueError: invalid literal for int() with base 10: ''

In [12]:
# run this cell to play the game
while imposter_alive:
    night_qc, vote_qc = init_qcs(num_players)
    init_night(night_qc, num_players)
    night_round(night_qc, num_players, imposter, alive_players)
    imposter_alive = who_died(night_qc, num_players, imposter, alive_players)
    if (check_game_over(alive_players, imposter)):
        break
    print("Players alive are ", end = "")
    for i in range(num_players):
        if alive_players[i] == 1:
            print(i, end = " ")
    print()
    vote(num_players, imposter, sabotages_left, alive_players)
    print(alive_players)
    if (check_game_over(alive_players, imposter)):
        break

print("game over")

{'110111': 1024}
Player 3 died
[1, 1, 1, 0, 1, 1]
Solution is 000100 with a count of 460
Player 2 has been killed! 
[1, 1, 0, 0, 1, 1]
{'111100': 517, '011111': 507}
Player 0 died
Player 1 died
[0, 0, 0, 0, 1, 1]
Solution is 100000 with a count of 66
Player 5 has been killed! 
[0, 0, 0, 0, 1, 0]
The imposter was killed!
Crewmates wins!
game over
